In [1625]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from crop_image import *
import imutils
from imutils import contours as imcnts
import skimage.io as io
%matplotlib inline

In [1626]:
image = cv2.imread('omr.png')
# cv2.imshow('image.jpg',image)
# cv2.waitKey(0)
number_of_choices=5
number_of_columns=3

In [1627]:
gray_scale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
# cv2.imshow('gray scale image.jpg',gray_scale)
# cv2.waitKey(0)

In [1628]:
"""gaussian = cv2.GaussianBlur(image,(3,3),0)
cv2.imshow('image',gaussian)
cv2.waitKey(0)"""

"gaussian = cv2.GaussianBlur(image,(3,3),0)\ncv2.imshow('image',gaussian)\ncv2.waitKey(0)"

In [1629]:
edged = cv2.Canny(gray_scale,50,200)
# cv2.imshow('edged.jpg',edged)
# cv2.waitKey(0)

In [1630]:
contours,_= cv2.findContours(edged, 
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

print("Number of Contours found = " + str(len(contours)))
  
cv2.drawContours(image, contours, -1,(0,255,0), 3)
  
# cv2.imshow('Contours.jpg', image)
# cv2.waitKey(0)
image = cv2.imread('omr.png')

Number of Contours found = 4


In [1631]:
#sorting all contours
if(len(contours)>0):
    contours=sorted(contours,key=cv2.contourArea,reverse=True)

In [1632]:
cv2.drawContours(image, contours, 0,(0,255,0), 3)
  
# cv2.imshow('Contours3.jpg', image)
# cv2.waitKey(0)
image = cv2.imread('omr.png')

In [1633]:
paper_contour=None
for cnt in contours:
    peri = cv2.arcLength(cnt,True)
    print(peri)
    # to approximate the given shape to polygon (high value leads to underestimation and low value leads to overestimation)
    ratio=0.02
    approx=cv2.approxPolyDP(cnt,ratio*peri,True)
    cv2.drawContours(image, [approx], 0,(0,0,255), 3)
    print(len(approx))
    coordinates=approx.ravel()
    for i in range(0,len(coordinates),2):
        x=coordinates[i]
        y=coordinates[i+1]
        cv2.putText(image, "vertex"+str(i//2), (x, y),cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0)) 
    if(len(approx)==4):
        paper_contour=approx
        break
# cv2.imshow('Contours2.jpg', image)
# cv2.waitKey(0)
image = cv2.imread('omr.png')

1906.0
4


In [1634]:
print(paper_contour.shape)

(4, 1, 2)


In [1635]:
paper= four_point_transform(image, paper_contour.reshape(4, 2))  
# cv2.imshow('paper', paper)
# cv2.waitKey(0)

In [1636]:
gray_scale_paper= four_point_transform(gray_scale, paper_contour.reshape(4, 2))  
# cv2.imshow('paper', gray_scale_paper)
# cv2.waitKey(0)

In [1637]:
# _,thresholded=cv2.threshold(gray_scale_paper,230,255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
# cv2.imshow('paper1', thresholded)
# cv2.waitKey(0)

In [1638]:
_,thresholded=cv2.threshold(gray_scale_paper,230,255, cv2.THRESH_BINARY_INV)
cv2.imshow('paper2', thresholded)
cv2.waitKey(0)

-1

In [1639]:
pap_cnts,_=cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [1640]:
def test_cnts(cnts):
    cv2.drawContours(thresholded, cnts, -1,255, 3)
  
    cv2.imshow('Contours.jpg', thresholded)
    cv2.waitKey(0)

In [1641]:
pre_question_cnts=[]
value=(8,9)
flag=True
for (i,cnt) in enumerate(pap_cnts):
    peri = cv2.arcLength(cnt,True)
    #print(peri)
    # to approximate the given shape to polygon (high value leads to underestimation and low value leads to overestimation)
    ratio=0.02
    approx=cv2.approxPolyDP(cnt,ratio*peri,True)
    ratio=0
    approx2=cv2.approxPolyDP(cnt,ratio*peri,True)
    print(len(approx2)-len(approx))
    (x,y,w,h)=cv2.boundingRect(cnt)
    aspect_ratio=w/h
    #print(len(approx))
    number_of_vertices=len(approx)
    if(len(approx2)-len(approx)>=10 and aspect_ratio>=0.9 and aspect_ratio<=1.1):
        pre_question_cnts.append(cnt)
        #print(len(approx))
        
print("length",len(pre_question_cnts))

5
6
2
22
10
12
12
14
14
17
4
4
4
14
14
12
12
10
7
4
6
23
8
6
4
4
6
15
10
12
12
14
14
10
15
15
15
12
14
7
21
12
12
12
12
16
10
10
10
8
10
12
14
14
14
14
11
1
11
28
14
12
12
12
11
12
14
14
14
14
14
15
15
17
14
11
7
14
10
12
12
14
14
17
4
4
4
4
13
13
13
10
16
27
20
14
13
16
16
16
16
16
4
4
4
6
8
8
6
8
15
18
16
5
12
12
10
10
9
9
11
11
8
13
13
17
16
17
17
23
9
17
16
16
17
17
11
15
12
10
12
11
10
14
14
16
16
21
6
17
12
12
14
14
16
13
4
4
4
6
13
13
14
14
10
1
2
2
2
9
10
14
16
16
16
13
4
4
4
6
13
11
12
12
12
5
2
7
2
15
6
6
3
0
14
11
0
5
4
16
11
15
7
16
10
10
10
10
14
12
12
12
8
10
13
14
16
15
21
3
2
9
8
4
4
18
18
16
16
8
8
12
12
10
10
15
17
18
4
7
4
4
16
16
14
13
10
10
16
16
17
14
8
8
14
27
10
3
7
9
9
9
12
11
14
15
15
12
14
15
15
13
12
15
19
14
6
10
13
13
6
3
6
13
12
12
10
10
12
12
12
12
14
16
10
12
14
14
16
16
4
4
16
16
6
11
11
12
10
10
22
17
2
8
5
5
17
11
14
14
14
15
16
17
16
7
10
16
21
22
7
16
11
14
10
10
13
13
13
10
13
16
17
16
13
15
21
27
7
15
13
12
14
14
8
8
14
14
8
15
13
14
12
12
27
1
3

In [1642]:
print(len(pre_question_cnts),len(pap_cnts))

240 394


In [1643]:
question_cnts,_=imcnts.sort_contours(pre_question_cnts,method='top-to-bottom')

In [1644]:
Answer_key= {0:1, 1:4, 2:0, 3:2}
correct=0
flag=False
cv2.drawContours(paper,question_cnts,0,(255,0,0), 1)
cv2.imshow('hgsvrgvj',paper)
cv2.waitKey(0)
cnt=question_cnts[0]
peri = cv2.arcLength(cnt,True)
#print(peri)
# to approximate the given shape to polygon (high value leads to underestimation and low value leads to overestimation)
ratio=0.00005
approx=cv2.approxPolyDP(cnt,ratio*peri,True)
(x,y,w,h)=cv2.boundingRect(cnt)
aspect_ratio=w/h
print(len(approx))
paper= four_point_transform(image, paper_contour.reshape(4, 2))  

23


In [1645]:
print(len(question_cnts))

240


In [1646]:
#test_cnts(question_cnts)

In [1647]:
# _,thresholded=cv2.threshold(gray_scale_paper,150,255, cv2.THRESH_BINARY_INV)
# pap_cnts,_=cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [1648]:
colors=[(255,0,0)
,(0,255,0)
,(0,0,255)
,(0,255,255)
,(255,255,0)
,(0,0,255)
,(0,255,255)
,(255,0,255)
,(192,192,192)
,(128,128,128)
,(128,0,0)
,(128,128,0)
,(0,128,0)
,(128,0,128)
,(0,128,128)
,(0,0,128),
       (255,215,0)]

In [ ]:
counter=0
qs=0
for (row,i) in enumerate(np.arange(0,len(question_cnts),number_of_columns*number_of_choices)):

    curr_row_cnts,_=imcnts.sort_contours(question_cnts[i:i+number_of_columns*number_of_choices])
    for k in np.arange(0,len(curr_row_cnts),number_of_choices):
        curr_ques_cnts=curr_row_cnts[k:k+number_of_choices]
        color1=colors[qs%len(colors)]
        cv2.drawContours(paper,curr_ques_cnts,-1,color1, 1)
        bubbled=None
        for (j,c) in enumerate(curr_ques_cnts):
            mask = np.zeros(thresholded.shape, dtype="uint8")
            #negative one at the last arguement to fill the contours
            cv2.drawContours(mask, [c], -1, 255, -1)
            mask= cv2.bitwise_and(thresholded, mask)
            total= cv2.countNonZero(mask)

            if bubbled is None or total >bubbled[0]:
                bubbled= (total, j)
            color= (0, 0, 255)
        print(f"qestion#{qs+1}: {chr(bubbled[1]+ord('A'))}")
        qs+=1
    counter+=1
print(counter)
counter=0
cv2.imshow('paper', paper)
cv2.waitKey(0)
paper= four_point_transform(image, paper_contour.reshape(4, 2)) 

qestion#1: C
qestion#2: D
qestion#3: A
qestion#4: B
qestion#5: A
qestion#6: A
qestion#7: C
qestion#8: C
qestion#9: D
qestion#10: E
qestion#11: C
qestion#12: A
qestion#13: E
qestion#14: E
qestion#15: D
qestion#16: C
qestion#17: D
qestion#18: B
qestion#19: B
qestion#20: D
qestion#21: A
qestion#22: A
qestion#23: B
qestion#24: D
qestion#25: A
qestion#26: A
qestion#27: D
qestion#28: A
qestion#29: D
qestion#30: C
qestion#31: A
qestion#32: C
qestion#33: A
qestion#34: A
qestion#35: A
qestion#36: C
qestion#37: A
qestion#38: E
qestion#39: C
qestion#40: B
qestion#41: C
qestion#42: D
qestion#43: B
qestion#44: C
qestion#45: E
qestion#46: C
qestion#47: D
qestion#48: A
16


In [ ]:
print(correct)
correct=0

In [ ]:
# cv2.imshow('Contours.jpg', thresholded)
# cv2.waitKey(0)

In [ ]:
for i,cnt in enumerate(question_cnts):
    color1=(255,0,0) if i%2==0 else (0,0,255)
    cv2.drawContours(paper,[cnt],-1,color1, 1)
# cv2.imshow('paper', paper)
# cv2.waitKey(0)
paper= four_point_transform(image, paper_contour.reshape(4, 2)) 

In [ ]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()



In [ ]:
print(contours)

In [ ]:
f"adham{chr(bubbled[1]+ord('A'))}"